In [1]:
import xmlrpc.client

import numpy as np
import pandas as pd


# Login

In [3]:
url = "https://swobbee1.odoo.com"
db = "swobbee1-out29"
username = "t.loe@swobbee.com"
password = "Haha2812!"

In [4]:
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
uid = common.authenticate(db,username,password, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

In [5]:
common.version()

{'server_version': '15.0+e',
 'server_version_info': [15, 0, 0, 'final', 0, 'e'],
 'server_serie': '15.0',
 'protocol_version': 1}

# creating domain and fields

In [6]:

domain = [('display_type', 'not in', ('line_section', 'line_note')), ("account_id.user_type_id.type", "in", ("receivable", "payable")), ("move_id.state", "=", "posted")]

fields = ["move_name", "date", "debit", "credit", "amount_currency", "currency_id", "name", "partner_id"]

items = models.execute_kw(db, uid, password, 'account.move.line', 'search_read', [domain, fields])

items = [{**item, 'currency_id': item["currency_id"][1], 'partner_id': item["partner_id"][1]} for item in items]

df = pd.DataFrame.from_dict(items)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df

df.groupby(["partner_id", "currency_id"])[["debit","credit","amount_currency"]].sum()


,,debit,credit,amount_currency
partner_id,currency_id,,,
2FSolutions SASU,EUR,4962.00,1830.00,3.132000e+03
AHOI Velo Cargobikes,EUR,417.69,92.82,3.248700e+02
AUDI AG I/FU-431,EUR,8126.04,8126.04,4.547474e-13
AVO mobility GmbH,EUR,10948.60,7350.04,3.598560e+03
AWO Kreisverband Emden e.V.,EUR,1302.14,99.84,1.202300e+03
Ansmann AG,EUR,41108.61,37783.15,3.325460e+03
Audi AG,EUR,15853.11,15496.11,3.570000e+02
Audi AG I/FU - 41,EUR,23700.95,4740.19,1.896076e+04
Automotive Safety Technologies GmbH,EUR,5616.80,5616.80,0.000000e+00


In [7]:
order = [{
    'partner_id': 1008,
    'sale_order_template_id': 2,
    'order_line': [
        (0, 0, {
            'product_uom_qty': 1,
            'product_id': 9180,
        }),
    ],
}]

id = models.execute_kw(db, uid, password, 'sale.order', 'create', order)

id

280

# Writing sales order lines based on quotation template didnt work


In [8]:

models.execute_kw(db, uid, password, 'sale.order', 'write', [id, 
                 {
                  'sale_order_template_id': 2   
                 }]
                 )


True

# hardcode Quotation template 1

In [14]:
id = models.execute_kw(db, uid, password, 'sale.order', 'create', 
                  [{
                      'partner_id': 1008,
                      'order_line': [
                          (0, 0, {
                              'display_type': 'line_section',
                              'name': 'Waren'
                          }),
                          (0, 0, {
                              'product_uom_qty': 1,
                              'product_id': 9180
                          }),
                          (0, 0, {
                              'product_uom_qty': 1,
                              'product_id': 8945
                          }),
                          (0, 0, {
                              'display_type': 'line_section',
                              'name': 'Abo'
                          }),
                          (0, 0, {
                              'display_type': 'line_note',
                              'name': '	Miete vom XX.XX.202X bis XX.XX.202X.'
                          }),
                          (0, 0, {
                              'display_type': 'line_section',
                              'name': 'Extra'
                          }),
                          (0, 0, {
                              'display_type': 'line_note',
                              'name': 'Service Pauschale vom XX.XX.202X bis XX.XX.202X. = 5€ / Monat / Akku'
                          }),
                          (0, 0, {
                              'product_uom_qty': 1,
                              'product_id': 10770
                          }),
                          (0, 0, {
                              'display_type': 'line_section',
                              'name': 'Transport'
                          }),
                          (0, 0, {
                              'display_type': 'line_note',
                              'name': 'Transport von Gefahrgüter'
                          }),
                          (0, 0, {
                              'product_uom_qty': 1,
                              'product_id': 10145
                          })
                      ]
                  }])

id

283

# write and then deleteSales order 

In [13]:

models.execute_kw(db, uid, password, 'sale.order', 'write', [[id], {
    'order_line': [
        (0, 0, {
                              'display_type': 'line_section',
                              'name': 'Waren'
                          }),
                          (0, '_', {
                              'product_uom_qty': 1,
                              'product_id': 9180
                          }),
                          (0, '_', {
                              'product_uom_qty': 1,
                              'product_id': 8945
                          }),
                          (0, '_', {
                              'display_type': 'line_section',
                              'name': 'Abo'
                          }),
                          (0, '_', {
                              'display_type': 'line_note',
                              'name': '	Miete vom XX.XX.202X bis XX.XX.202X.'
                          }),
                          (0, '_', {
                              'display_type': 'line_section',
                              'name': 'Extra'
                          }),
                          (0, '_', {
                              'display_type': 'line_note',
                              'name': 'Service Pauschale vom XX.XX.202X bis XX.XX.202X. = 5€ / Monat / Akku'
                          }),
                          (0, '_', {
                              'product_uom_qty': 1,
                              'product_id': 10770
                          }),
                          (0, '_', {
                              'display_type': 'line_section',
                              'name': 'Transport'
                          }),
                          (0, '_', {
                              'display_type': 'line_note',
                              'name': 'Transport von Gefahrgüter'
                          }),
                          (0, '_', {
                              'product_uom_qty': 1,
                              'product_id': 10145
                          })
    ]
}])

models.execute_kw(db, uid, password, 'sale.order', 'unlink', [[id]])


True

# Creating Invoice from Sales Order

In [16]:
context = {
    "active_model": 'sale.order',
    "active_ids": [id],
    "active_id": id,
}

wizard_id = models.execute_kw(db, uid, password, 'sale.advance.payment.inv', 'create', [{'advance_payment_method': 'delivered'}], {'context': context})

models.execute_kw(db, uid, password, 'sale.advance.payment.inv', 'create_invoices', [[wizard_id]], {'context': context})

{'type': 'ir.actions.act_window_close'}

# test

In [18]:
ids = models.execute_kw(db, uid, password, 
                             'sale.order', 'search', [[['id', '=', '266']]] )

ids

[266]

In [19]:
record = models.execute_kw(db,uid,password,
                            'sale.order.template', 'read', [2])

record

for item in record:
    print("Name: {}\nID: {}\n".format(item['name'],item['sale_order_template_line_ids']))

Name: BaaS - DE - GP
ID: [1, 17, 18, 2, 3, 4, 5, 6, 7]



In [20]:
models.execute_kw(db, uid, password, 'sale.order.template', 'search_read', [[['active', '=', True]]], {'fields': ['name', 'sale_order_template_line_ids'], 'limit': 5})


[{'id': 1, 'name': 'Default Template', 'sale_order_template_line_ids': []},
 {'id': 2,
  'name': 'BaaS - DE - GP',
  'sale_order_template_line_ids': [1, 17, 18, 2, 3, 4, 5, 6, 7]},
 {'id': 3,
  'name': 'BaaS - ENG - GP',
  'sale_order_template_line_ids': [10, 21, 22, 11, 12, 13, 14, 38, 15, 16]},
 {'id': 4, 'name': 'Yamaha - Baqme', 'sale_order_template_line_ids': []}]

In [21]:
partner_id = 1008
product_ids = [9180,9137,9134]

id = models.execute_kw(db, uid, password, 'sale.order', 'create', [{
    'partner_id': partner_id,
    'order_line': [(0,0, {'product_id': product_id}) for product_id in product_ids]
}])


